#### Sold Property Links Scrape (Non Texas)

In [ ]:
import pandas as pd
import csv
from datetime import datetime
import zipwiseproperties
import os
import logging
from concurrent.futures import ThreadPoolExecutor, as_completed

# Configure logging to write to a file and to the console
log_filename = f'logger/sold_property_scraper_{datetime.now().strftime("%Y-%m-%d")}.log'
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler(log_filename),
        logging.StreamHandler()
    ]
)

def save_links_to_csv(links, filename):
    """
    Save a list of links to a CSV file.

    Parameters:
    links (list): List of property links.
    filename (str): The name of the CSV file to save the links to.
    """
    with open(filename, 'a', newline='') as file:
        writer = csv.writer(file)
        for link in links:
            writer.writerow([link])

def scrape_properties(zip_code, base_url):
    """
    Scrape property links for a given zip code from the base URL.

    Parameters:
    zip_code (str): The zip code to scrape properties for.
    base_url (str): The base URL to start scraping from.

    Returns:
    list: A list of unique property links for the zipcode.
    """
    all_links = []
    try:
        html, error = zipwiseproperties.fetch_html_with_zyte(base_url)
        if not html:
            logging.error(f"Failed to fetch HTML for {zip_code}")
            return all_links

        # Parse initial page
        links = zipwiseproperties.parse_properties(html)
        if links:
            all_links.extend(links)
            logging.info(f"Added {len(links)} links from base URL")

        total_pages_count = zipwiseproperties.total_pages(html)
        print(f"{zip_code} has {total_pages_count} total pages")
        logging.info(f"{zip_code} has {total_pages_count} total pages")

        # If there are multiple pages, scrape them
        if total_pages_count >= 20:
            price_filters = [
                {"min": 0, "max": 200000},
                {"min": 200000, "max": 250000},
                {"min": 250000, "max": 300000},
                {"min": 300000, "max": 325000},
                {"min": 325000, "max": 350000},
                {"min": 350000, "max": 375000},
                {"min": 375000, "max": 400000},
                {"min": 400000, "max": 425000},
                {"min": 425000, "max": 450000},
                {"min": 450000, "max": 475000},
                {"min": 475000, "max": 500000},
                {"min": 500000, "max": 550000},
                {"min": 550000, "max": 600000},
                {"min": 600000, "max": 800000},
                {"min": 800000, "max": None}  # 800000+ (no max filter needed)
            ]
            bed_filters = [
                (0, 0),  # Studio
                (1, 1),  # 1 Bed
                (2, 2),  # 2 Beds
                (3, 3),  # 3 Beds
                (4, 4),  # 4 Beds
                (5, None)  # 5+ Beds (no max filter needed)
            ]

            for price_filter in price_filters:
                filtered_url = zipwiseproperties.update_url_with_price(base_url, price_filter['min'], price_filter['max'])
                # print(read_url(filtered_url))

                logging.info(f"Adding links from price filter {price_filter['min']}-{price_filter['max']}")
                for min_beds, max_beds in bed_filters:
                    bed_filtered_url = zipwiseproperties.update_url_with_beds(filtered_url, min_beds, max_beds)
                    # print(read_url(bed_filtered_url))
                    html, error = zipwiseproperties.fetch_html_with_zyte(bed_filtered_url)
                    pages_count = zipwiseproperties.total_pages(html)
                    links = zipwiseproperties.parse_properties(html)
                    if links:
                        all_links.extend(links)
                    logging.info(f"Added {len(links)} links from beds filter {min_beds}-{max_beds}")
                    all_links.extend(scrape_all_pages(zip_code, bed_filtered_url, pages_count, min_beds, max_beds))
        else:
            all_links.extend(scrape_all_pages(zip_code, base_url, total_pages_count))

    except Exception as e:
        logging.error(f"Error processing {zip_code}: {e}")
    # Remove duplicates before returning
    unique_links = list(set(all_links))
    logging.info(f"Total unique links scraped for {zip_code}: {len(unique_links)}")
    
    return unique_links

def scrape_all_pages(zip_code, base_url, total_pages_count, min_beds=None, max_beds=None):
    """
    Scrape all pages for a given zip code and URL.

    Parameters:
    zip_code (str): The zip code to scrape properties for.
    base_url (str): The base URL to start scraping from.
    total_pages_count (int): The total number of pages to scrape.
    min_beds (int, optional): Minimum number of beds for filtering.
    max_beds (int, optional): Maximum number of beds for filtering.

    Returns:
    list: A list of property links.
    """
    links = []
    for page_number in range(2, total_pages_count + 1):
        url = zipwiseproperties.update_url_with_page(base_url, page_number)
        html, error = zipwiseproperties.fetch_html_with_zyte(url)
        if html:
            page_links = zipwiseproperties.parse_properties(html)
            if page_links:
                links.extend(page_links)
                logging.info(f"Added {len(page_links)} links from page {page_number} "
                             f"for {zip_code} with bed filter ({min_beds}-{max_beds})")
    return links
    

def process_zip_code(zip_code, filename):
    """
    Process a single zip code and save property links to a CSV file.

    Parameters:
    zip_code (str): The zip code to process.
    filename (str): The name of the CSV file to save the links to.
    """
    logging.info(f"Processing zip code: {zip_code}")
    base_url = zipwiseproperties.generate_zipcode_url_sold(zip_code)
    links = scrape_properties(zip_code, base_url)
    if links:
        save_links_to_csv(links, filename)
        logging.info(f"Saved {len(links)} links for zip code: {zip_code}")

def process_msa(msa_name, msa_zipcodes, filename):
    """
    Process all zip codes in an MSA and save property links to a CSV file.

    Parameters:
    msa_name (str): The name of the MSA.
    msa_zipcodes (list): List of zip codes in the MSA.
    filename (str): The name of the CSV file to save the links to.
    """
    # Ensure directory exists
    os.makedirs(os.path.dirname(filename), exist_ok=True)
    
    with open(filename, 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['Property Link'])

    # Process zip codes in parallel
    with ThreadPoolExecutor(max_workers=14) as executor:
        futures = [executor.submit(process_zip_code, zip_code, filename) for zip_code in msa_zipcodes]
        for future in as_completed(futures):
            try:
                future.result()  # Wait for each future to complete
            except Exception as e:
                logging.error(f"Error processing zip code {futures[future]}: {e}")

    # Remove duplicates
    df_links = pd.read_csv(filename)
    df_links.drop_duplicates(inplace=True)
    df_links.to_csv(filename, index=False)
    logging.info(f"Removed duplicates. Total unique links saved: {len(df_links)}")

def main():
    """
    Main function to read zip codes from a CSV file and scrape property links for specified MSAs.
    """
    zipcodes_df = pd.read_csv('zip_codes_by_msa.csv', dtype={'GEOID_ZCTA5_20': str})
    zipcodes_df['GEOID_ZCTA5_20'] = zipcodes_df['GEOID_ZCTA5_20'].str.zfill(5)

    msas = [
        "Boston-Cambridge-Newton, MA-NH"
        ]
    
    today = datetime.now().strftime("%Y-%m-%d")
    for msa in msas:
        print(msa)
        filename = f'sold_properties_links/{today}/{msa.replace(",", "").replace("-", " ").lower()}_properties.csv'
        msa_zipcodes = zipcodes_df[zipcodes_df['CBSA Title_x'] == msa]['GEOID_ZCTA5_20'].unique()
        logging.info(f"Starting scraping for {msa}")
        start_time = datetime.now()
        process_msa(msa, msa_zipcodes, filename)
        end_time = datetime.now()
        logging.info(f"Time taken for scraping {msa}: {end_time - start_time}")

# Execute
if __name__ == "__main__":
    main()


##### JSON File Extracter

In [ ]:
import json
import csv
import pandas as pd
from bs4 import BeautifulSoup
from datetime import datetime
import logging
from concurrent.futures import ThreadPoolExecutor, as_completed
import zipwiseproperties
from queue import Queue
import os

# Configure logging
log_filename = f'logger/json_scraper_{datetime.now().strftime("%Y-%m-%d")}.log'
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler(log_filename),
        logging.StreamHandler()
    ]
)

BATCH_SIZE = 500  # Write to CSV every 100 URLs processed


def create_directory(city):
    today = datetime.now().strftime("%Y-%m-%d")
    # base_directory = os.getcwd()
    base_directory = '/scratch/kota.sha'  # Set the base directory to the desired location
    directory = os.path.join(base_directory, 'sold_properties_results', city, today)
    os.makedirs(directory, exist_ok=True)
    return directory
  
def process_url(url, sl_no):
    try:
        html, error = zipwiseproperties.fetch_html_with_zyte(url)
        if not html:
            return url, error  # Return the error for logging

        soup = BeautifulSoup(html, 'html.parser')
        data_script = soup.find('script', id='__NEXT_DATA__')
        data_json = json.loads(data_script.string)
        sold_data = json.loads(data_json['props']['pageProps']['componentProps']['gdpClientCache'])
        
        first_key = next(iter(sold_data))
        first_element_value = sold_data[first_key]

        # Convert the first element into JSON format
        json_data = json.dumps(first_element_value, indent=4)
        
        return sl_no, url, json_data  # Return the data for saving

    except Exception as e:
        return url, str(e)  # Return error message

def flush_to_csv(queue, file_path, headers):
    """
    Write a batch of results or errors to the CSV file.
    """
    with open(file_path, 'a', newline='') as file:
        writer = csv.writer(file)
        while not queue.empty():
            writer.writerow(queue.get())

def scrape_city(city, csv_filename, max_workers):
    directory = create_directory(city)
    json_dir = os.path.join(directory, 'jsonfiles')
    os.makedirs(json_dir, exist_ok=True)

    results_file = f'{directory}/{city}_sold_property_details.csv'
    error_file = f'{directory}/error_sold_property_urls.csv'

    start_time = datetime.now()
    logging.info(f"Scraping started at: {start_time} for {city}")

    # Load property links into a Pandas DataFrame for easier manipulation
    df = pd.read_csv(csv_filename)
    property_links = df.iloc[:, 0].tolist()  # Assuming the first column contains URLs

    logging.info(f"Total URLs to process: {len(property_links)}")

    # Load already processed URLs and their serial numbers from the results file
    processed_urls = set()
    processed_sl_nos = set()  # To track processed serial numbers
    last_sl_no = 0
    if os.path.exists(results_file):
        processed_df = pd.read_csv(results_file)
        processed_urls = set(processed_df['URL'].tolist())  # Assuming 'URL' is the column name
        processed_sl_nos = set(processed_df['Serial Number'].tolist())  # Track serial numbers
        last_sl_no = max(processed_sl_nos) if processed_sl_nos else 0
        logging.info(f"Found {len(processed_urls)} already processed URLs. Skipping them.")

    # Filter out already processed URLs
    remaining_urls = [(sl_no + 1, url) for sl_no, url in enumerate(property_links) if url not in processed_urls]
    logging.info(f"Remaining URLs to process: {len(remaining_urls)}")

    success_queue = Queue()  # Thread-safe queues for collecting success and error results
    error_queue = Queue()

    # Prepare the results CSV headers if the results file does not exist
    if not os.path.exists(results_file):
        with open(results_file, 'w', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(['Serial Number', 'URL', 'JSON File Path'])

    if not os.path.exists(error_file):
        with open(error_file, 'w', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(['URL', 'Error'])

    processed_count = 0

    # Use ThreadPoolExecutor for concurrent URL processing
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        future_to_url = {executor.submit(process_url, url, sl_no): (sl_no, url) for sl_no, url in remaining_urls}
        for future in as_completed(future_to_url):
            result = future.result()
            processed_count += 1

            if len(result) == 3:
                # Success: Write the JSON data to a file
                sl_no, url, json_data = result
                json_file_path = os.path.join(json_dir, f'{sl_no}.json')  # JSON file named after sl_no
                with open(json_file_path, 'w') as json_file:
                    json_file.write(json_data)
                success_queue.put((sl_no, url, json_file_path))  # Add to success queue
            else:
                # Error: Log the error URL
                url, error = result
                error_queue.put((url, error))  # Add to error queue

            # Every BATCH_SIZE URLs, write results and errors to CSV
            if processed_count % BATCH_SIZE == 0:
                flush_to_csv(success_queue, results_file, ['Serial Number', 'URL', 'JSON File Path'])
                flush_to_csv(error_queue, error_file, ['URL', 'Error'])
                logging.info(f"Processed {processed_count} URLs so far...")

    # Final flush for any remaining results or errors
    flush_to_csv(success_queue, results_file, ['Serial Number', 'URL', 'JSON File Path'])
    flush_to_csv(error_queue, error_file, ['URL', 'Error'])

    end_time = datetime.now()
    duration = end_time - start_time

    logging.info(f"All remaining property links processed for {city}. Time taken for scraping: {duration.total_seconds()} seconds")
    logging.info(f"All property details saved to {results_file}")
    logging.info(f"URLs with errors saved to {error_file}")

if __name__ == "__main__":
    logging.info("Scraping session started")
    total_start_time = datetime.now()

    cities_info = [
        ("boston","sold_properties_links/2024-09-04/boston cambridge newton ma nh_properties.csv",14)
    ]

    for city, csv_filename, max_workers in cities_info:
        scrape_city(city, csv_filename, max_workers)

    total_end_time = datetime.now()
    total_duration = total_end_time - total_start_time
    logging.info(f"Total scraping session completed in {total_duration.total_seconds()} seconds")

     